# Исследование надежности заемщиков

**Описание проекта**

Заказчик — кредитный отдел банка.

Необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**

* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов;
* total_income — ежемесячный доход;
* purpose — цель получения кредита.

## Общая информация о данных

Импортируем библиотеку pandas:

In [ ]:
import pandas as pd

Прочитаем данные из csv-файла и сохраним в переменную `data`:

In [ ]:
data = pd.read_csv('bank_data.csv')

Выведем первые 10 строчек датафрейма `data` на экран:

In [ ]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`:

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Обработка пропусков

Выведем количество пропущенных значений для каждого столбца:

In [ ]:
data.isna().sum()

,0
children,0
days_employed,2174
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


В датафрейме есть пропущенные значения в столбцах `days_employed` и `total_income`, значит необходимо обработать пропуски.

Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`:

In [ ]:
for i in data['income_type'].unique():
    data.loc[(data['income_type'] == i) & (data['total_income'].isna()), 'total_income'] = data.loc[(data['income_type'] == i), 'total_income'].median()
data.isna().sum()

,0
children,0
days_employed,2174
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


В столбце `days_employed` содержатся артефакты -  отрицательное количество дней трудового стажа. Заменим все отрицательные значения положительными с помощью метода `abs()`:

In [ ]:
data['days_employed'] = data['days_employed'].abs()
data['days_employed']

,days_employed
0,8437.673028
1,4024.803754
2,5623.422610
3,4124.747207
4,340266.072047
...,...
21520,4529.316663
21521,343937.404131
21522,2113.346888
21523,3112.481705


Выведем медианное значение трудового стажа `days_employed` в днях:

In [ ]:
data.groupby('income_type')['days_employed'].median()

,days_employed
income_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,2689.368353
компаньон,1547.382223
пенсионер,365213.306266
предприниматель,520.848083
сотрудник,1574.202821
студент,578.751554


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`:

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения: -1 и 20. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`:

In [ ]:
data = data.loc[(data['children'] >= 0) & (data['children'] < 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`:

In [ ]:
for i in data['income_type'].unique():
    data.loc[(data['income_type'] == i) & (data['days_employed'].isna()), 'days_employed'] = data.loc[(data['income_type'] == i), 'days_employed'].median()
data.isna().sum()

,0
children,0
days_employed,0
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`:

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education` -  приведем к нижнему регистру:

In [ ]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных:

In [ ]:
dup = data.duplicated().sum()
dup

71

Удалим дубликаты:

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

Создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [ ]:
def categorize_income(income):
    if 0 <= income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

Применим функцию:

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`:

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Cоздадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [ ]:
def categorize_purpose(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образовани' in purpose:
        return 'получение образования'
    return 'операции с недвижимостью'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследовательский анализ

### Проверим зависимость между количеством детей и возвратом кредита в срок

Построим сводную таблицы для определения доли должников относительно всех заемщиков по количеству детей в семьях

In [ ]:
children_pivot = data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])
# переименуем столбцы
children_pivot = children_pivot.rename(columns={'count':'Всего заемщиков', 'sum':'Всего должников', 'mean':'Доля должников, %'})
# выведем дол должников в процентах и округлим
children_pivot['Доля должников, %'] = (children_pivot['Доля должников, %'] * 100).round(3)
children_pivot

,Всего заемщиков,Всего должников,"Доля должников, %"
,debt,debt,debt
children,,,
0,14091,1063,7.544
1,4808,444,9.235
2,2052,194,9.454
3,330,27,8.182
4,41,4,9.756
5,9,0,0.000


Создадим функцию для категоризации данных на две группы

In [ ]:
def categorized_children(row):
    if row == 0:
        return 'Семьи без детей'
    return 'Семьи с детьми'

Применим функцию для создания нового столбца:

In [ ]:
data['children_category'] = data['children'].apply(categorized_children)

Построим сводную таблицу по категориям семей:

In [ ]:
categorized_children_pivot = data.pivot_table(index=['children_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
# переименуем столбцы
categorized_children_pivot = categorized_children_pivot.rename(columns={'count':'Всего заемщиков', 'sum':'Всего должников', 'mean':'Доля должников, %'})
# выведем долю должников по категориям семей в процентах и округлим
categorized_children_pivot['Доля должников, %'] = (categorized_children_pivot['Доля должников, %'] * 100).round(3)
categorized_children_pivot

,Всего заемщиков,Всего должников,"Доля должников, %"
,debt,debt,debt
children_category,,,
Семьи без детей,14091,1063,7.544
Семьи с детьми,7240,669,9.240


**Вывод:** при первом рассмотрении данных можно прийти к выводу, что возврат кредитов в срок не зависит от количества детей. Попытка категоризации данных на две группы показала такой же результат. Однако, данное сравнение не совсем корректное, так как размер выборок отличается почти в 2 раза. Для достоверных результатов необходимо применение статистических методов сравнения.

Если не брать в расчет семьи с 3-5 детьми, то по первой таблице видно, что процент должников увеличивается по мере увеличения детей в семье. Можно сделать вывод, что самыми надежными плательщикам являются семьи без детей.

### Проверим зависимость между семейным положением и возвратом кредита в срок

Построим сводной таблицы для определения доли должников относительно всех заемщиков по семейному положению:

In [ ]:
family_status_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count', 'sum', 'mean'])
# переименуем столбцы
family_status_pivot = family_status_pivot.rename(columns={'count':'Всего заемщиков', 'sum':'Всего должников', 'mean':'Доля должников, %'})
# округлим кол-во должников по семейному положению
family_status_pivot['Всего должников'] = family_status_pivot['Всего должников'].round(3)
# выведем в процентах и округлим доли должников по семейному положению
family_status_pivot['Доля должников, %'] = (family_status_pivot['Доля должников, %'] * 100).round(3)
family_status_pivot

,Всего заемщиков,Всего должников,"Доля должников, %"
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,9.764
в разводе,1189,84,7.065
вдовец / вдова,951,63,6.625
гражданский брак,4134,385,9.313
женат / замужем,12261,927,7.561


**Вывод:** анализируя полученные результаты, видно, что процент должников колеблется от 6 до 9%. Наименьший процент у вдовцов/вдов, а наибольший - у людей, официально не состоящих в отношениях. Считается, что пара, узаконившая свои отношения, является болеее надежным плательщиком, что, в целом, подтверждается полученными данными. Однако, разный размер выборок также не позволяет сделать достоверный вывод.

### Проверим зависимость между уровнем дохода и возвратом кредита в срок

Построим сводную таблицу для определения доли должников относительно всех заемщиков по уровню дохода в семьях:

In [ ]:
total_income_category_pivot = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
# переименуем столбцы
total_income_category_pivot = total_income_category_pivot.rename(columns={'count':'Всего заемщиков', 'sum':'Всего должников', 'mean':'Доля должников, %'})
# округлим кол-во должников по уровню дохода в семье
total_income_category_pivot['Всего должников'] = total_income_category_pivot['Всего должников'].round(3)
# округлим и выведем в процентах доли должников по уровню дохода в семье
total_income_category_pivot['Доля должников, %'] = (total_income_category_pivot['Доля должников, %'] * 100).round(3)
total_income_category_pivot

,Всего заемщиков,Всего должников,"Доля должников, %"
,debt,debt,debt
total_income_category,,,
A,25,2,8.000
B,5014,354,7.060
C,15921,1353,8.498
D,349,21,6.017
E,22,2,9.091


**Вывод:** исходя из полученных данных, невозможно провести грамотный анализ, так как размер выборок не поддается сравнению.

Если рассмотреть самые многочисленные группы - уровни дохода B и C - видно, что люди с более высоким доходом являются более надежными плательщиками, так как процент должников на 1,5% меньше.

### Проверим влияение разных целей кредита на его возврат в срок

Построим сводную таблицу для определения доли должников относительно всех заемщиков по целям кредита в семьях:

In [ ]:
purpose_category_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
# переименуем столбцы
purpose_category_pivot = purpose_category_pivot.rename(columns={'count':'Всего заемщиков', 'sum':'Всего должников', 'mean':'Доля должников, %'})
# округлим кол-во должников относительно всех заемщиокв по целям кредита
purpose_category_pivot['Всего должников'] = purpose_category_pivot['Всего должников'].round(3)
# округлим и выведем доли должников относительно всех заемщиокв по целям кредита
purpose_category_pivot['Доля должников, %'] = (purpose_category_pivot['Доля должников, %'] * 100).round(3)
purpose_category_pivot

,Всего заемщиков,Всего должников,"Доля должников, %"
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,9.348
операции с недвижимостью,10751,780,7.255
получение образования,3988,369,9.253
проведение свадьбы,2313,183,7.912


**Вывод:** исходя из имеющихся данных, можно сделать вывод, что чаще всего люди берут кредит на покупку или ремонт недвижимости, так как это наиболее дорогостоящие операции. В целом, процент должников по всем операциям примерно равен и варьируется в диапазоне 7-9%, а лучше всего выплачивается кредит при опреациях с недвижимостью и проведении свадеб.

## Общий вывод

На стадии предобработки данных были обнаружены пропущенные значения в столбцах "стаж работы" и "доход", а в столбце "стаж работы" и "кол-во детей" были обнаружены артефакты - значения стажа работы были отрицательнымм, некоторые значения кол-ва детей были аномальными для выборки. Пропуски в столбцах были заменены медианным значением, а артефекаты исправлены. Также была выполнена категоризация столбцов "доход" и "цель кредита" для последующего анализа.

В начале проекта была поставлена задача выяснить влияение семейного положения и количества детей на факт погашения кредита в срок.

В результате исследования выявилась следующая закономерность: с ростом числа детей увеличвается процент должников по кредитам - с 7,5% для семей без детей до 9,4% для семей с 2 детьми. Семьи с 3 и более детей были исключены из анализа, так как малые выборки не позволяют сделать грамотное сравнение. Разница в проценте должников между семьями с 1 ребенком и 2 детьми составляет 0,2%, что может быть погрешностью или следствием разного размера выборок. Таким образом, можно сделать вывод, что наличие детей существенно увеливает процент должников, а значит, семьи без детей являются наиболее надежными плательщиками.

Анализируя зависимость семейного положения на возврат кредита в срок, можно увидеть, что у вдов и вдовцов, а также у разведенных людей, наименьший процент должников, но этими данными можно пренебречь из-за малого размера выборки. Самый большой размер выборки и одновременно самый низкий процент должников - 7,5%, что на 1,8 и на 2,2% ниже по сравнению с остальными категориями - у людей в браке, а значит, эта категория людей с большей вероятность вернет кредит в срок.

Также были проанализированы зависимости уровня дохода и цели кредита на возврат кредита в срок.
Выяснилось, что люди с более высоким доходом (от 200 тысяч рублей до 1 млн. рублей) реже нарушают сроки кредита, чем люди с доходом от 50 до 200 тысяч рублей. Операции с недвижимостью и проведение свальбы являются самыми надежными (около 7% должников) по сравнению с операциями с автомобилем или кредитом на получение образования (около 9% должников).

Так как размер выборок для всех исследований сильно отличался, необходимо провести более глубокий анализ данных с применением статистических методов для получения более достоверной картины из-за различного размера выборок.